In [1]:
import sys
import time
sys.path.append('../shared')
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import copy
import elbo_calculator as ec
import train
import argparse
import os
import model_hi_vae as mhv
from scipy.io import loadmat
%load_ext autoreload

In [2]:
column_types=[0,0,1,3,2,0]
input_dim = 9
hidden_dim =3
latent_dim=4
s_dim=2
x_dim = 9

In [3]:
rows = 100

In [4]:
x0 = np.random.normal(1,2,size=(rows,1))
x1 = np.random.normal(0,1,size=(rows,1))
x2 = np.log(np.random.poisson(1.5,size=(rows,1))+ 0.01)
x3 = tf.one_hot(np.random.randint(0,3,size=rows),3).numpy()
x4 = tf.one_hot(np.random.randint(0,2,size=rows),2).numpy()
x5 = np.random.normal(0.5,1,size=(rows,1))

In [5]:
x = tf.concat([x0,x1,x2,x3,x4,x5], axis=-1).numpy().astype(np.float32)

In [6]:
x[0]

array([ 2.9434335 , -0.11920778,  0.00995033,  0.        ,  0.        ,
        1.        ,  1.        ,  0.        ,  0.6792717 ], dtype=float32)

In [7]:
model = mhv.model_hi_vae(input_dim, hidden_dim, latent_dim, s_dim, column_types)

In [8]:
z = model.encoder(x)

In [9]:
gen_z_func =model.get_z_generator()

In [10]:
zs =gen_z_func(x)
zs, s_prop, beta, gamma= zs

In [11]:
tf.reshape(zs,(-1, zs.shape[-1]))

<tf.Tensor: id=1408, shape=(20000, 4), dtype=float32, numpy=
array([[ 1.9866936 ,  2.4067202 ,  0.85983616, -1.9537601 ],
       [ 0.2117889 , -0.16883129,  0.4506566 , -1.5115532 ],
       [-1.0516093 ,  1.4522104 ,  0.20814729,  2.0010996 ],
       ...,
       [ 2.2869532 ,  0.91702753, -2.819608  ,  0.47290552],
       [ 1.9900264 ,  2.7436357 ,  0.4184302 ,  0.2849459 ],
       [-1.2117223 ,  2.2490628 , -0.38700575,  0.43903273]],
      dtype=float32)>

In [12]:
zs_reshaped = tf.reshape(zs,(-1, zs.shape[-1]))
output = model.decoder([
                zs_reshaped,
                beta,
                gamma])

In [13]:
zs_reshaped.shape

TensorShape([20000, 4])

In [14]:
log_p_xz_func = model.get_func_log_p_xz()
log_q_z_x_func = model.get_func_log_q_z_x()

In [15]:
zs.shape

TensorShape([2, 100, 100, 4])

In [16]:
secx = model.encoder(x)

In [17]:
secx[1]

<tf.Tensor: id=1820, shape=(2, 100, 4), dtype=float32, numpy=
array([[[ 2.68327296e-02,  8.97384286e-02,  9.10049736e-01,
          1.03859141e-01],
        [ 2.37559170e-01, -4.28190351e-01,  5.22212148e-01,
          7.88778901e-01],
        [-6.36598945e-01,  8.03027034e-01,  1.38725030e+00,
          4.16047275e-01],
        [ 2.07266271e-01,  1.41821063e+00,  5.97280860e-01,
          1.96705073e-01],
        [ 3.16092104e-01, -7.63149858e-01,  6.25799417e-01,
          3.30644667e-01],
        [-3.45600545e-01,  1.66888487e+00,  8.88115168e-01,
          8.63509119e-01],
        [ 4.24623877e-01,  1.38936400e+00,  4.19329077e-01,
          1.09257445e-01],
        [ 2.46442780e-01,  1.20921075e-01,  5.04546762e-01,
          6.81339025e-01],
        [ 1.80365294e-01,  1.01271999e+00,  7.79765010e-01,
         -1.75860047e-01],
        [ 7.80261755e-02,  1.12457418e+00,  8.46086025e-01,
         -1.06164806e-01],
        [ 1.39557779e-01, -6.45827770e-01,  6.71917915e-01,
        

In [18]:
ebc = ec.elbo_calculator()

In [19]:
loss_func = ebc.get_loss_func_2()

In [20]:
loss_func(model, x)

IndexError: too many indices for array

In [23]:
output[:,1]

TypeError: list indices must be integers or slices, not tuple